In [3]:
"""
Extract only the essential data needed for the Streamlit bike redistribution map.
"""

import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import math

# ==================================================================================
# CONFIGURATION (same as your Streamlit app)
# ==================================================================================
TOP_N = 300
TARGET_STATIONS_PER_ZONE = 12
INPUT_FILE = r"C:\Users\magia\OneDrive\Desktop\NY_Citi_Bike\2.Data\Prepared Data\nyc_2022_essential_data.csv"
OUTPUT_DIR = r"C:\Users\magia\OneDrive\Desktop\NY_Citi_Bike\2.Data\Prepared Data\aggregated"

df = pd.read_csv(
    INPUT_FILE,
    low_memory=False,
    on_bad_lines="skip",
    encoding_errors="ignore"
)

print(f"Loaded {len(df):,} trip records")

# ==================================================================================
# STEP 1: Extract Station Metadata (top 300 stations with coords and zones)
# ==================================================================================
print("\n[1/2] Extracting station metadata...")

# Convert datetime columns
for col in ["started_at", "ended_at"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Get station coordinates (median lat/lon per station)
starts_geo = (df.dropna(subset=["start_station_name", "start_lat", "start_lng"])
              .groupby("start_station_name")[["start_lat", "start_lng"]]
              .median()
              .rename(columns={"start_lat": "lat", "start_lng": "lon"}))

ends_geo = (df.dropna(subset=["end_station_name", "end_lat", "end_lng"])
            .groupby("end_station_name")[["end_lat", "end_lng"]]
            .median()
            .rename(columns={"end_lat": "lat", "end_lng": "lon"}))

stations_all = starts_geo.combine_first(ends_geo)
stations_all.index.name = "station"

# Get station volumes
starts_count = df.groupby("start_station_name").size().rename("starts")
ends_count = df.groupby("end_station_name").size().rename("ends")
station_volume = (pd.concat([starts_count, ends_count], axis=1)
                  .fillna(0)
                  .assign(total=lambda x: x.starts + x.ends)
                  .sort_values("total", ascending=False))

# Keep top N stations
top_stations = (station_volume.head(TOP_N)
                .join(stations_all, how="left")
                .dropna(subset=["lat", "lon"])
                .copy())

# Cluster into geographic zones
n_clusters = max(1, math.ceil(len(top_stations) / TARGET_STATIONS_PER_ZONE))
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=256, n_init="auto")
top_stations["geo_zone"] = kmeans.fit_predict(top_stations[["lat", "lon"]])

# Renumber zones 0, 1, 2, ... for consistency
zone_mapping = {old: new for new, old in enumerate(sorted(top_stations["geo_zone"].unique()))}
top_stations["geo_zone"] = top_stations["geo_zone"].map(zone_mapping).astype(int)

# Save station metadata
# The index contains station names, so we need to reset it and rename properly
stations_metadata = top_stations.reset_index()
stations_metadata = stations_metadata[[stations_metadata.columns[0], "lat", "lon", "geo_zone", "total"]]
stations_metadata.columns = ["station_name", "lat", "lon", "geo_zone", "total_trips"]
stations_metadata.to_csv(f"{OUTPUT_DIR}/stations_metadata.csv", index=False)
print(f"✓ Saved stations_metadata.csv ({len(stations_metadata)} stations)")

# ==================================================================================
# STEP 2: Extract Hourly Flows per Zone
# ==================================================================================
print("\n[2/2] Extracting hourly flows...")

station_to_zone = top_stations["geo_zone"].to_dict()

# Calculate hourly starts per zone
starts_hourly = (df.loc[df["start_station_name"].isin(top_stations.index)]
                 .dropna(subset=["started_at"])
                 .assign(zone=lambda x: x["start_station_name"].map(station_to_zone),
                         hour=lambda x: x["started_at"].dt.hour,
                         dow=lambda x: x["started_at"].dt.dayofweek,
                         month=lambda x: x["started_at"].dt.to_period("M").astype(str))
                 .groupby(["month", "dow", "hour", "zone"]).size()
                 .rename("starts").reset_index())

# Calculate hourly ends per zone
ends_hourly = (df.loc[df["end_station_name"].isin(top_stations.index)]
               .dropna(subset=["ended_at"])
               .assign(zone=lambda x: x["end_station_name"].map(station_to_zone),
                       hour=lambda x: x["ended_at"].dt.hour,
                       dow=lambda x: x["ended_at"].dt.dayofweek,
                       month=lambda x: x["ended_at"].dt.to_period("M").astype(str))
               .groupby(["month", "dow", "hour", "zone"]).size()
               .rename("ends").reset_index())

# Create complete grid (all months × days × hours × zones, even if zero trips)
all_months = sorted(set(starts_hourly["month"]) | set(ends_hourly["month"]))
all_dows = sorted(set(starts_hourly["dow"]) | set(ends_hourly["dow"]))
all_hours = range(24)
all_zones = sorted(top_stations["geo_zone"].unique())

grid = pd.MultiIndex.from_product(
    [all_months, all_dows, all_hours, all_zones],
    names=["month", "dow", "hour", "zone"]
).to_frame(index=False)

# Merge with actual data and fill missing with zeros
hourly_flows = (grid.merge(starts_hourly, on=["month", "dow", "hour", "zone"], how="left")
                .merge(ends_hourly, on=["month", "dow", "hour", "zone"], how="left")
                .fillna({"starts": 0, "ends": 0}))

# Save hourly flows
hourly_flows.to_csv(f"{OUTPUT_DIR}/hourly_flows.csv", index=False)
print(f"✓ Saved hourly_flows.csv ({len(hourly_flows)} records)")

# ==================================================================================
# SUMMARY
# ==================================================================================
print("\n" + "="*80)
print("EXTRACTION COMPLETE!")
print("="*80)

import os
stations_size = os.path.getsize(f"{OUTPUT_DIR}/stations_metadata.csv") / 1024
flows_size = os.path.getsize(f"{OUTPUT_DIR}/hourly_flows.csv") / (1024 * 1024)

print(f"\nGenerated files:")
print(f"  1. stations_metadata.csv - {stations_size:.1f} KB")
print(f"  2. hourly_flows.csv - {flows_size:.1f} MB")
print(f"\nTotal size: {stations_size/1024 + flows_size:.2f} MB (from 5 GB!)")
print(f"Reduction: {100 * (1 - (stations_size/1024 + flows_size) / 5000):.2f}%")

Loaded 29,838,166 trip records

[1/2] Extracting station metadata...


C:\Users\magia\anaconda3\envs\CB22\lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


✓ Saved stations_metadata.csv (300 stations)

[2/2] Extracting hourly flows...
✓ Saved hourly_flows.csv (50400 records)

EXTRACTION COMPLETE!

Generated files:
  1. stations_metadata.csv - 15.2 KB
  2. hourly_flows.csv - 1.3 MB

Total size: 1.33 MB (from 5 GB!)
Reduction: 99.97%


In [5]:
stations_metadata.to_csv('stations_metadata.csv', index = False)

In [6]:
hourly_flows.to_csv('hourly_flows.csv', index = False)